# Deep belief network for bimodal embedding

In [1]:
import timeit
import pandas as pd
import numpy as np
import theano
from array import array

# Local package
from DBN import DBN


Using gpu device 0: GRID K520


In [2]:
numpy_rng = np.random.RandomState(123)

# construct the Deep Belief Network
dbn = DBN(numpy_rng=numpy_rng, n_ins=50, hidden_layers_sizes=[200, 100, 50], n_outs=10)

In [3]:
prefvectors = pd.read_csv('prefvectors.csv')

In [4]:
from array import array
X = prefvectors.factors.apply(lambda x: np.array(eval(x)))
shared_x = theano.shared(np.asarray(list(X), dtype=theano.config.floatX), borrow=True)

In [12]:
batch_size = 20
pretraining_epochs = 5
batch_count = shared_x.get_value(borrow=True).shape[0] / batch_size

pretraining_fns = dbn.pretraining_functions(train_set_x=shared_x, batch_size=batch_size, k=10)

start_time = timeit.default_timer()

## Pre-train layer-wise
for i in xrange(dbn.n_layers):
    # go through pretraining epochs
    for epoch in xrange(pretraining_epochs):
        # go through the training set
        c = []
        for batch_index in xrange(batch_count):
            c.append(pretraining_fns[i](index=batch_index, lr=0.01))
        print 'Pre-training layer %i, epoch %d, cost ' % (i, epoch),
        print np.mean(c)

end_time = timeit.default_timer()

Pre-training layer 0, epoch 0, cost  -5.94614
Pre-training layer 0, epoch 1, cost  -0.179251
Pre-training layer 0, epoch 2, cost  -0.0943452
Pre-training layer 0, epoch 3, cost  -0.065069
Pre-training layer 0, epoch 4, cost  -0.0493488
Pre-training layer 1, epoch 0, cost  -155.28
Pre-training layer 1, epoch 1, cost  -144.023
Pre-training layer 1, epoch 2, cost  -141.61
Pre-training layer 1, epoch 3, cost  -140.474
Pre-training layer 1, epoch 4, cost  -139.855
Pre-training layer 2, epoch 0, cost  -8.39875
Pre-training layer 2, epoch 1, cost  -2.22004
Pre-training layer 2, epoch 2, cost  -2.05128
Pre-training layer 2, epoch 3, cost  -1.97654
Pre-training layer 2, epoch 4, cost  -1.93933


/usr/local/lib/python2.7/dist-packages/theano/scan_module/scan_perform_ext.py:133: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility
  from scan_perform.scan_perform import *


In [47]:
output = dbn.sigmoid_layers[2].params[0].get_value(borrow=True)

In [50]:
output.dump("dbnout.pickle")